In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
miss_val = df.isnull().sum()
miss_val

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('NA')
    else:
        df[col] = df[col].fillna(0.0)

In [7]:
print(df.isnull().sum())

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64


In [8]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [10]:
df['industry'].mode()[0]

'retail'

In [11]:
cols = [
    'interaction_count', 
    'lead_score', 
    'number_of_courses_viewed', 
    'annual_income'
]
correlation_matrix = df[cols].corr()
print(correlation_matrix)

                          interaction_count  lead_score  \
interaction_count                  1.000000    0.009888   
lead_score                         0.009888    1.000000   
number_of_courses_viewed          -0.023565   -0.004879   
annual_income                      0.027036    0.015610   

                          number_of_courses_viewed  annual_income  
interaction_count                        -0.023565       0.027036  
lead_score                               -0.004879       0.015610  
number_of_courses_viewed                  1.000000       0.009770  
annual_income                             0.009770       1.000000  


In [12]:
pairs = [
    ('interaction_count', 'lead_score'),
    ('number_of_courses_viewed', 'lead_score'),
    ('number_of_courses_viewed', 'interaction_count'),
    ('annual_income', 'interaction_count'),
]

correlations = {
    pair: abs(correlation_matrix.loc[pair[0], pair[1]])
    for pair in pairs
}
max_corr_pair = max(correlations, key=correlations.get)
print("The pair with the biggest correlation is:", max_corr_pair, "with value:", correlations[max_corr_pair])

The pair with the biggest correlation is: ('annual_income', 'interaction_count') with value: 0.027036472404814417


In [13]:
X = df.drop('converted', axis=1)
y = df['converted']

In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

In [15]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [16]:
cat_cols = ['industry', 'location', 'lead_source', 'employment_status']

In [17]:
mi_scores = {}
for col in cat_cols:
    X_encoded = X_train[col].astype('category').cat.codes.values.reshape(-1, 1)
    score = mutual_info_classif(X_encoded, y_train, discrete_features=True, random_state=42)
    mi_scores[col] = round(score[0], 2)
mi_scores
max(mi_scores, key=mi_scores.get)

'lead_source'

In [18]:
cat_cols = ['industry', 'location', 'lead_source', 'employment_status']

X_train_enc = pd.get_dummies(X_train, columns=cat_cols)
X_val_enc = pd.get_dummies(X_val, columns=cat_cols)

X_val_enc = X_val_enc.reindex(columns=X_train_enc.columns, fill_value=0)

In [19]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_enc, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [20]:
val_preds = model.predict(X_val_enc)
acc = accuracy_score(y_val, val_preds)
rounded_acc = round(acc, 2)
print("Validation accuracy:", rounded_acc)

Validation accuracy: 0.74


In [21]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_enc, y_train)
val_pred = model.predict(X_val_enc)
acc_full = accuracy_score(y_val, val_pred)

In [22]:
features_to_test = ["industry", "employment_status", "lead_score"]
diffs = {}

for feature in features_to_test:
    cols_to_drop = [col for col in X_train_enc.columns if col.startswith(feature + "_")]
    X_train_subset = X_train_enc.drop(columns=cols_to_drop)
    X_val_subset = X_val_enc.drop(columns=cols_to_drop)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_subset, y_train)
    val_pred = model.predict(X_val_subset)
    acc_subset = accuracy_score(y_val, val_pred)
    diffs[feature] = abs(acc_full - acc_subset)
    diffs
min(diffs, key=diffs.get)

'industry'

In [23]:
C_values = [0.01, 0.1, 1, 10, 100]
results = {}

for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_enc, y_train)
    val_preds = model.predict(X_val_enc)
    acc = accuracy_score(y_val, val_preds)
    results[C] = round(acc, 3)

results

{0.01: 0.743, 0.1: 0.743, 1: 0.743, 10: 0.743, 100: 0.743}

In [24]:
best_C = min([key for key in results if results[key] == max(results.values())])
best_C

0.01